In [1]:
!pip install pydrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# 1. Authenticate and create the PyDrive client.
def authenticateWithGDrive():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  return GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 1.2MB/s 
  Running setup.py bdist_wheel for pydrive ... - done
  Stored in directory: /content/.cache/pip/wheels/ea/bd/b1/14e948c56cce2c7ac1589346cb54d7efb659fa2ddfa6abba88
Successfully built pydrive


In [0]:
def gDriveAuthorization(auth_name,save_dir):
  auth.authenticate_user()
  gauth = GoogleAuth()
  # Try to load saved client credentials
  authcred_dir=save_dir+"/"+auth_name+".txt"
  
  if gauth.credentials is None:
      if os.path.isfile(authcred_dir):
        gauth.LoadCredentialsFile(authcred_dir)
      # Authenticate if they're not there
      gauth.credentials = GoogleCredentials.get_application_default()
      gauth.SaveCredentialsFile(authcred_dir)
  elif gauth.access_token_expired:
      # Refresh them if expired
      gauth.Refresh()
      gauth.SaveCredentialsFile(authcred_dir)
  else:
      # Initialize the saved creds
      gauth.Authorize()
  
  return GoogleDrive(gauth)


In [0]:
# Retrieving from folder.
def retrieveFilesFromGoogleDrive(parentfolder_id,subfolder_name) :
  file_list = drive.ListFile({'q': "'{parent_id}'".format(parent_id=parentfolder_id) + " in parents and trashed=false"}).GetList()
  for file1 in file_list:
    if file1['title'] == subfolder_name :
     model_dir = drive.ListFile({'q': "'{folderid}' in parents and trashed=false".format(folderid=file1['id'])}).GetList()
     for file2 in model_dir:
        drive.CreateFile({'id': file2['id']}).GetContentFile(file2['title']) 
        
#Recursive function to walk through google drive repo        
def retrieveFolderfromGoogleDrive(parentfolder_id,targetdir) :
  drive=authenticateWithGDrive()
  file_list = drive.ListFile({'q': "'{parent_id}'".format(parent_id=parentfolder_id) + " in parents and trashed=false"}).GetList()
  for file1 in file_list:
    #print(file1['title'] +'__'+file1['mimeType'])
    sub_dir1=targetdir+'/'+file1['title']
    #if folder call same function
    if '.git' not in file1['title'] :
      if file1['mimeType'] == "application/vnd.google-apps.folder":
          if not os.path.isdir(sub_dir1):
            !mkdir {sub_dir1}
          retrieveFolderfromGoogleDrive(file1['id'],sub_dir1)
      else  :
        if not os.path.isfile(sub_dir1):
          drive.CreateFile({'id': file1['id']}).GetContentFile(sub_dir1)


def uploadFileToGoogleDrive(local_file_dir,name,parentfolder_id):
  drive=gDriveAuthorization("skeletal-insights-creds",".")
  file=drive.CreateFile({'title': name,'parents':[{"kind": "drive#fileLink", "id": parentfolder_id}]})
  file.SetContentFile(local_file_dir)
  file.Upload()
  return file.get('id')

#Loop through GDrive folder to run gen_coord.py on images 
#Save generated coordinates CSVs to a file in Data/Processed subfolder

def generateCoordinatesGoogleDrive(GoogleDriveParentFolderId,outputCSV='empty.csv'):
  if ('.csv' in outputCSV):
    !cp template.csv $outputCSV
  #get list of contents in the parentfolder
  drive=authenticateWithGDrive()
  
  file_list = drive.ListFile({'q': "'{parent_id}'".format(parent_id=GoogleDriveParentFolderId) + " in parents and trashed=false"}).GetList()
  for file1 in file_list:
    drive=gDriveAuthorization("skeletal-insights-creds",".")
    #if subfolder with yogapose names then initiate csv file with pose name
    if file1['mimeType'] == "application/vnd.google-apps.folder":
      outputCSV = file1['title']+".csv"
      generateCoordinatesGoogleDrive(file1['id'],outputCSV)
    else :  
    #if image file then try to process as images
      try:
        if ('.png' in file1['title'] or '.jpg' in file1['title']):
          tmp_file_dir = './' + file1['title']
          #print(tmp_file_dir)
          gFile1=drive.CreateFile({'id': file1['id']})
          gFile1.GetContentFile(tmp_file_dir)
          !TF_CUDNN_USE_AUTOTUNE=0 python demo/generate_coord.py "$tmp_file_dir" $outputCSV
          !rm "$tmp_file_dir"
          #gFile1.Trash()  # Move file to trash.
      except:
        print('Error for file : ' + file1['title'])
        
  #Once done reauthenticate and copy csv to GoogleDrive Parent Folder 
  uploadFileToGoogleDrive(outputCSV,outputCSV,GoogleDriveParentFolderId)

def printIdsFromGoogleDrive(parentfolder_id) :
  drive=authenticateWithGDrive()
  file_list = drive.ListFile({'q': "'{parent_id}'".format(parent_id=parentfolder_id) + " in parents and trashed=false"}).GetList()
  for file1 in file_list:
    print(file1['title'] +' : '+file1['id'])

In [4]:
printIdsFromGoogleDrive('0B4AG0JVNcFVOZE14R3BTbzVmMm8')

Data Distribution: Current State : 1GECFNoDeW353nZGdQlo0bmedZAMcHTR41ESfR_IZySo
project : 1avPwX24tvAiE0z3p91p1z372rAZE5yA4
archive : 1ma2Ai3yMCzAKfHgI9CbQ9bAAvG8N-pDM
scripts : 0B8yWoVxMY3IVY1Zfb0NWUVZDSG8
deepercut_results : 0B4AG0JVNcFVOMHJNdmVxWFRDb2c
Jay's : 0B4AG0JVNcFVOc3dieWdkaFByS2M


In [5]:
printIdsFromGoogleDrive('1llU86VZPIwW_QAWy6uZKlELgxDZTCt2q')

train : 1PdtepkXFNc5KpwrSamsD3z2gVJWHZ8p3
validation : 1VuYuU49u7zg7nIXmxY9WOH5XeRNkm90g
test : 1LZ91H2OQpcRh-Ya6mgJ-vLNgvCOXRJL2
yoga_train_pkl : 1vQhQXjy4X9woILOTXeY3C-tz14xJkBQE
deeper_cut_output : 1C6cAgc19n3F9i77FC9GKLWMNe4AsJYTB


In [6]:
printIdsFromGoogleDrive('1Ay8W74_15gHTpEz-SARe-mxXBE-yTKoe')

resnet56_skeletal_insights : 1-QajaKkqBnju9xRFuRl0zutSfrhZ_4dA
resnet56_skeletal_insights : 1RnKFyaq-DNFnAZoOrpUZP2IHM2hOY-rj
resnet32_skeletal_insights : 1u86nJVvMeY7OVLZDlNLUr2Tqjknpb91F
conv_nn_v03 : 1fIjFTYqp2Gp7tLpnfM9eiUWPx15jKwXv
cifar10_cnn_pretrain : 1HkFAZy6cAPPlYw15_mD9bEjClIjNuZaH
pose-tensorflow : 1gSnQCscjEAp884a8VrxfE7rzM25lRa1X
pretrain : 154BQuWxOLLuQ9RzbEuEs-F4Q7mTG36df
tflearn-CIFAR_v0.3.tfl : 1jedDj_D1z2QGLz67GPowZAzKrAtlcb3X
resnet56_skeletal_insights : 1KgB_x8qdVLy-YLD9NtjOwcAw14SAh0z6


In [0]:
!mkdir pose-tensorflow

In [0]:
retrieveFolderfromGoogleDrive('1gSnQCscjEAp884a8VrxfE7rzM25lRa1X','./pose-tensorflow')

In [9]:
!pip install easydict
!apt-get install python3-tk -y
!pip install scikit-neuralnetwork

  Running setup.py bdist_wheel for easydict ... - done
  Stored in directory: /content/.cache/pip/wheels/06/13/9e/3af761786e5b5b8c41b8a151da64915ddcd18bb464135f9551
Successfully built easydict
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  blt libtcl8.6 libtk8.6 libxext6 libxss1 tk8.6-blt2.5 tzdata x11-common
Suggested packages:
  blt-demo tcl8.6 tk8.6 tix python3-tk-dbg
The following NEW packages will be installed:
  blt libtcl8.6 libtk8.6 libxext6 libxss1 python3-tk tk8.6-blt2.5 tzdata
  x11-common
0 upgraded, 9 newly installed, 0 to remove and 1 not upgraded.
Need to get 2,445 kB of archives.
After this operation, 12.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/main amd64 libxext6 amd64 2:1.3.3-1 [29.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful/main amd64 x11-common all 1:7.7+19ubuntu3 [22.0 kB]
Get:3 http://archive.ubuntu.com/

    100% |████████████████████████████████| 2.8MB 448kB/s 
  Running setup.py bdist_wheel for scikit-neuralnetwork ... - done
  Stored in directory: /content/.cache/pip/wheels/b1/80/10/c479c0c019e9834647b9973754acd8dc2d69a00b833eb5d161
  Running setup.py bdist_wheel for Lasagne ... - \ done
  Stored in directory: /content/.cache/pip/wheels/d5/3f/25/2bf9c38906847df05e3529e5df230e349ac5a44b953f1f233b
  Running setup.py bdist_wheel for Theano ... - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/46/a2/7d/b4cac381d5151daa9f9e0b3e4e4b65edaea6355ae296c97cf2
Successfully built scikit-neuralnetwork Lasagne Theano


In [0]:
import os
os.chdir('pose-tensorflow')

In [0]:
!PYTHONPATH="${PYTHONPATH}:/content/pose-tensorflow"
!export PYTHONPATH

In [12]:
!pip install --ignore-installed --upgrade https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-1.1.0-cp36-cp36m-linux_x86_64.whl

    100% |████████████████████████████████| 31.4MB 45kB/s 
    100% |████████████████████████████████| 6.4MB 208kB/s 
    100% |████████████████████████████████| 17.2MB 80kB/s 
    100% |████████████████████████████████| 327kB 3.8MB/s 
    100% |████████████████████████████████| 51kB 9.7MB/s 
    100% |████████████████████████████████| 491kB 2.2MB/s 


In [13]:
!python -c 'import tensorflow as tf; print(tf.__version__)'

1.1.0


In [0]:
#Generate for train
generateCoordinatesGoogleDrive('1PdtepkXFNc5KpwrSamsD3z2gVJWHZ8p3','.');

2018-02-19 21:47:49.299810: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:47:49.299870: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:47:49.299882: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:47:49.299893: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:47:49.299907: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't

2018-02-19 21:49:11.521576: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:49:11.521617: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:49:11.521629: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:49:11.521639: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:49:11.521653: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't

2018-02-19 21:50:33.872320: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:50:33.872380: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:50:33.872397: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:50:33.872413: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:50:33.872427: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't

2018-02-19 21:51:56.124591: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:51:56.124637: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:51:56.124649: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:51:56.124669: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:51:56.124683: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't

2018-02-19 21:53:18.596392: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:53:18.596465: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:53:18.596480: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:53:18.596495: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 21:53:18.596524: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't

...

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-02-19 22:00:16.085430: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:00:16.085477: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:00:16.085490: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:00:16.085500:

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-02-19 22:01:39.852260: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:01:39.852309: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:01:39.852322: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:01:39.852331:

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-02-19 22:03:03.468404: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:03:03.468454: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:03:03.468466: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:03:03.468475:

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-02-19 22:04:26.169477: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:04:26.169532: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:04:26.169545: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:04:26.169555:

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-02-19 22:05:49.812622: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:05:49.812670: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:05:49.812684: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-02-19 22:05:49.812700:

In [22]:
!ls -l


total 328
-rw-r--r-- 1 root root 25269 Feb 19 14:11 adho_mukha_svanasana.csv
-rw-r--r-- 1 root root 29712 Feb 19 14:11 bharadvajasana.csv
-rw-r--r-- 1 root root  1317 Feb 19 14:11 config.py
-rw-r--r-- 1 root root  1523 Feb 19 14:11 config.pyc
drwxr-xr-x 3 root root  4096 Feb 19 14:11 dataset
-rw-r--r-- 1 root root   821 Feb 19 14:11 default_config.py
-rw-r--r-- 1 root root  1171 Feb 19 14:11 default_config.pyc
drwxr-xr-x 5 root root  4096 Feb 19 14:11 demo
-rw-r--r-- 1 root root   177 Feb 19 14:11 gen_coordinates.sh
-rw-r--r-- 1 root root  7651 Feb 19 14:11 LICENSE
drwxr-xr-x 3 root root  4096 Feb 19 14:11 matlab
drwxr-xr-x 5 root root  4096 Feb 19 14:12 models
drwxr-xr-x 3 root root  4096 Feb 19 14:12 nnet
-rw-r--r-- 1 root root 23554 Feb 19 14:11 pasasana.csv
-rw-r--r-- 1 root root  1050 Feb 19 14:11 pose_1.csv
-rw-r--r-- 1 root root  5838 Feb 19 14:11 pose_coordinates.csv
-rw-r--r-- 1 root root  1050 Feb 19 14:11 pose.csv
drwxr-xr-x 2 root root  4096 Feb 19 14:13 __pycache__
-rw-r--

In [0]:
#Generate for validation
generateCoordinatesGoogleDrive('1VuYuU49u7zg7nIXmxY9WOH5XeRNkm90g','.')

In [0]:
#Generate for test
#generateCoordinatesGoogleDrive('1LZ91H2OQpcRh-Ya6mgJ-vLNgvCOXRJL2','.')

In [0]:
#!python -c "import nnet"